In [6]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
#인스타그램 로그인
driver = webdriver.Chrome(".../.../chromedriver.exe")
url = 'https://www.instagram.com'
driver.get(url)
time.sleep(10)
email = '여기에 아이디 입력'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)
password = '여기에 비밀번호 입력'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(5)
#검색어를 입력하여 해당 페이지로 이동
def insta_search(word):
    url = 'https://www.instagram.com/explore/tags/' + word
    return url 
url = insta_search('영화')
driver.get(url)
time.sleep(50)
#검색어에 해당하는 첫번째 게시글 선택
def select_first(driver):
    first = driver.find_element_by_css_selector('div._9AhH0')
    first.click()
    time.sleep(5)
select_first(driver)

import re, unicodedata
#게시글 정보를 가져오는 함수
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    try:
        content = soup.select('div.C4VMK > span')[0].text  
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^s#,\\]+',content)
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    data = [content, date, like, place, tags] 
    return data 
#다음 게시글 이어서 조회하는 함수
def move_next(driver): # 다음 게시글 조회
    right = driver.find_element_by_css_selector('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

results = [ ] 
target = 100 #인스타그램 게시글 100개 탐색하기
for i in range(target):
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)   
    except:
        time.sleep(2)
        move_next(driver)

import pandas as pd 
results_df = pd.DataFrame(results)
results_df.columns = ['content', 'date', 'like', 'place', 'tags']
results_df.to_csv('./insta_movie_crawling.csv',index=False)#csv파일로 저장

WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
